# USCyber Games
## Season III - 2023

### Archiveception [Forensics]
1. Untar start_layer.gz archive yields two files
2. Bzip2
3. 7z yields password protected 7z archive and the password in a plaintext file
4. 7z extracts the flag.txt file  
**Flag:** `uscg_open{d0wNw4rd_15_7h3_0n1y_w4y_f0rw4rD}`

### Metadata [Forensics]
Strings yields the string comment with the flag in plaintext.  
**Flag:** `flag{334_shards_of_hail_fall_ominously}`

### Word [Forensics]
String is hidden in error image as metadata tag.  
**Flag:** `USCG{parliamentary_carpet}`

### Viginere [Crypto]
Just reversing the Vigneere encryption and running it in reverse 1000 times.  
**Flag:** `uscg{morerepetitionsdoesntmeansafer}`

In [ ]:
#Vigenere

def decrypt(key, ciphertext):
    plaintext = [chr(((ord(c)-ord(k))%26)+97) for c,k in zip(ciphertext,key)]
    return "".join(plaintext)

key = "raaojpkaufmsrez"
flag = "tmjuefrwtvjirfx"
for i in range(1000):
    key = decrypt(flag, key)
    flag = decrypt(key, flag)
print(key)
print(flag)

### The First Digram Substitution Cipher [Crypto]
The cipher is a Digram Substitution Cipher, the playfair cipher.  Bruteforcing is easy for the little keyspace.
Generating all keys, while doing some backtracking when encountering errors seemed to the trick.  Yielding a subset of the plaintext set, which was searchable.

**Flag:** `uscg{THIS_CIPHER_WAS_ONCE_REJECTED_FOR_ITS_PERCEIVED_COMPLEXITY}`

In [ ]:
import string
from pycipher import Playfair

secret = "SRJEEUJLTCDWWEIETCSINEOSFGTLMEOJTCENJUWNLEPICOUKFR".lower()
KEY = list("CH_R_ESWTO_BD__IJ__P_____".lower())
REM = [val for val in string.ascii_lowercase if val not in KEY and val != "q"]

def rec(key, alph):
    if "_" not in key:
        decoded = Playfair("".join(key)).decipher(secret)
        if decoded.startswith("THISCIPHERWASO"):
            print("".join(key), decoded)
            return 0
        return 5
    mod_idx = key.index("_")
    for val in alph:
        if val in key:
            continue
        key[mod_idx] = val
        ret = rec(key, alph)
        key[mod_idx] = "_"
        if ret != 0:
            return ret-1
    return 0
rec(KEY, REM)

### Table Tennis
The PCAP file contains a Ping Pong session.  However, the "Window Size Scaling factor" appears to be harbouring regular ASCII data.  
These packets are only sent from the user with the `ping` messages, and words like `ls`, `base64` and `flag` appears.

Extracting all packages to text format with all contents, We can filter out all rows, not starting with the `00 30` byte index, and continue fiddling down the text until we get something like below which, when concatenating the two first chars of each line becomes, `ls flag base64 dXNjZ3tMMG9rMW5HX3RocjB1Z0hfVGgzX3cxbkRvV30=`, and translates into `uscg{L0ok1nG_thr0ugH_Th3_w1nDoW}`

```
lsW..........8..

fl]..........9..
agb..........9..

baa..........:..
sePr.........:..
64...........;..

dX_^.........<..
NjuB.........<..
Z3in.........=..
tMOI.........=..
MGvE.........=..
9r...........=..
MWv..........>..
5H.$.........>..
X3k..........?..
Rop..........?..
cj_..........?..
B1...........@..
Z0i..........@..
hfZ..........@..
VGl..........A..
gz[..........A..
X3j..........A..
cx_..........B..
bk`..........B..
Rvpu.........C..
V3l..........C..
0=...........C..
```


`qMtStnD3Q-1e3-yYXCKOHcPagm-qiGkcnJcDuzOoac8`

### Semaphore
The conversation contains a bunch of erronuous TCP packets from A to B.  
Looking at the packets, we see the TCP flags appears to be readable, ASCII values.  
Exporting the entire conversation to JSON, then filtering out the TCP flag values, and grabbing the last byte of each we get a sequence of byte values, which converts into ASCII characters.

After converting, concatenating and decoding the bytearray using Base64, we get a text with the flag

**Flag:** `USCG{s3map4h0r3_tcp_f1ag5}`

In [ ]:
#Semaphore

import json
import base64
with open("semaphore.json") as jsonfile:
    data = json.load(jsonfile)
content = bytearray()
for packet in data:
    tcp_flag_str = packet["_source"]["layers"]["tcp"]["tcp.flags"][-2:]
    tcp_flag_int = int(tcp_flag_str, 16)
    tcp_flag_byte = bytes.fromhex(tcp_flag_str)
    content.append(tcp_flag_int)
print(content)
print()
print(base64.b64decode(content))

### Rumble In The Jumble

The file is encoded using a substitution cipher for the key space `[A-Za-z]`.  By analysing a standard ELF file we can generate a partial translation table which is shown below.  
```
Key space: abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
Translate: pbrisgm.ydfaeohvxl.c..tnzuF.CGPJ.R.BM.ES.TLNDU.QI...
```

In the file we find the flag string which needs to be deciphered.
```
00002000: 0100 0200 0000 0000 446e 6e6a 2046 6e62  ........Dnnj Fnb
00002010: 2120 506f 6d20 6b72 6c66 2064 653a 2054  ! Pom krlf de: T
00002020: 4e43 447b 6b31 7133 6a5f 776f 335f 757a  NCD{k1q3j_wo3_uz
00002030: 6738 7233 357d 0000 011b 033b 2c00 0000  g8r35}.....;,...
```

Using the transposition table we get
```
Dnnj Fnb! Pom krlf de: TNCD{k1q3j_wo3_uzg8r35}
Good Job! The flag is: USCG{f1x3d_th3_jum8l35}
```
